Repo card metadata block was not found. Setting CardData to empty.


Test
---- <fsspec.implementations.local.LocalFileSystem object at 0x7f1a66c90500>


Saving the dataset (0/1024 shards):   0%|          | 0/70000 [00:00<?, ? examples/s]

Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
Exception ignored in:   File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
<generat

FileNotFoundError: [Errno 2] No such file or directory

: generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
<generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>Traceback (most recent call last):

  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)    queue.put(result)

RuntimeError

RuntimeError: generator ignored GeneratorExit: 
generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_utils.py", line 679, in _write_generator_to_queue
    queue.put(result)
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Dataset._save_to_disk_single at 0x7f1a65d3b350>
Traceback (most recent call last):
  File "/home/jovyan/shares/SR003.nfs2/micromamba/envs/bm/lib/python3.12/site-packages/datasets/utils/py_